In [111]:
# Importação e configuração
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

In [152]:
# Carregando arquivo Csv

df = pd.read_csv('base_case.csv', sep='\t')

### Avaliando o conjunto de dados de forma macro

In [153]:
# Visualizar os Primeiros Registros
df.head()

,index,alvo,REF_DATE,PRODUTO,DIAS_ATRASO,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,col_50,col_51,col_52,col_53,col_54,col_55,col_56,col_57,col_58,col_59,col_60,col_61,col_62,col_63,col_64,col_65,col_66,col_67,col_68,col_69,col_70,col_71,col_72,col_73,col_74,col_75,col_76,col_77,col_78,col_79,col_80,col_81,col_82,col_83,col_84,col_85,col_86,col_87,col_88,col_89,col_90,col_91,col_92,col_93,col_94,col_95,col_96,col_97,col_98,col_99,col_100,col_101,col_102,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,col_112,col_113,col_114,col_115,col_116,col_117,col_118,col_119,col_120,col_121,col_122,col_123,col_124,col_125,col_126,col_127,col_128,col_129,col_130,col_131,col_132,col_133,col_134,col_135,col_136,col_137,col_138,col_139,col_140,col_141,col_142,col_143,col_144,col_145,col_146,col_147,col_148,col_149,col_150,col_151,col_152,col_153,col_154,col_155,col_156,col_157,col_158,col_159,col_160,col_161,col_162,col_163,col_164,col_165,col_166,col_167,col_168,col_169,col_170,col_171,col_172,col_173,col_174,col_175,col_176,col_177,col_178,col_179,col_180,col_181,col_182,col_183,col_184,col_185,col_186,col_187,col_188,col_189,col_190,col_191,col_192,col_193,col_194,col_195,col_196,col_197,col_198,col_199,col_200,REGIAO
0,0,0,2023-01-31,CREDITO PESSOAL,52,2461.11,30 a 60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.00,0.00,2.00,2.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,A,A,C,C,A,A,A,A,0,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,7,44.65,7.65,44.65,7.65,NaN,NaN,18.50,NaN,INDEFINIDO,INDEFINIDO,INDEFINIDO,NaN,NaN,NaN,4,0,0,0,7,4,0,0,0,7,6,0,0,0,9,0.00,NaN,0.00,0.00,NaN,SEM APROVACAO,SEM PASSAGEM,SEM APROVACAO,SEM APROVACAO,SEM PASSAGEM,S,N,44.65,NaN,N,N,VAREJO,S,35.70,N,VAREJO-BANCOS,N,NaN,N,APROV:S - NOVASOLIC:S - SOLICMESMOLOCAL:N,APROV:N - NOVASOLIC:N - SOLICMESMOLOCAL:N,0,0,0,0.00,SEM INFORMACAO,0.00,0,FUNDAMENTAL COMPLETO,BASICO,0,1.00,0,0,1,E,0,NaN,NaN,0,0,DE 25 A 39 ANOS,Y,NaN,0,0,0,0,0,0,2,NaN,MEDIO,NaN,3.00,-1.00,1.00,URBANO,NORTE
1,1,0,2022-12-29,CREDITO PESSOAL,37,4674.50,30 a 60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,A,A,A,C,A,A,A,C,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,7,41.61,15.47,41.61,15.47,31.82,30.66,6.50,0.60,INDEFINIDO,INDEFINIDO,INDEFINIDO,INDEFINIDO,INDEFINIDO,INDEFINIDO,3,2,0,0,7,3,2,0,0,7,3,2,0,0,7,NaN,NaN,0.67,0.67,NaN,SEM PASSAGEM,SEM PASSAGEM,SEM PASSAGEM,SEM PASSAGEM,SEM PASSAGEM,S,S,15.47,30.66,N,N,VAREJO,N,NaN,N,NaN,N,NaN,N,APROV:S - NOVASOLIC:N - SOLICMESMOLOCAL:N,APROV:S - NOVASOLIC:N - SOLICMESMOLOCAL:N,0,0,0,0.00,2 A 4 SM,1.00,1,SUPERIOR INCOMPLETO,MEDIO,0,1.00,0,0,1,D,0,NaN,NaN,0,0,DE 25 A 39 ANOS,Y,NaN,0,1,0,0,0,0,4,D,MEDIO,D,2.00,1.00,1.00,URBANO,NORDESTE
2,2,0,2023-09-29,CREDITO PESSOAL,36,43649.17,30 a 60,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,B,B,B,B,A,A,A,A,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,5,69.55,0.82,69.55,0.82,50.62,50.62,4.00,NaN,INDEFINIDO,INDEFINIDO,INDEFINIDO,INDEFINIDO,INDEFINIDO,INDEFINIDO,2,2,0,0,6,2,2,0,0,6,2,2,0,0,6,1.00,NaN,NaN,1.00,NaN,ALTA,SEM PASSAGEM,SEM PASSAGEM,ALTA,SEM PASSAGEM,S,S,0.82,50.62,S,N,BANCOS,N,NaN,N,NaN,N,NaN,N,APROV:S - NOVASOLIC:N - SOLICMESMOLOCAL:N,APROV:S - NOVASOLIC:N - SOLICMESMOLOCAL:N,1,0,0,0.00,4 A 10 SM,1.00,1,SUPERIOR COMPLETO,ALTO,1,4.00,1,0,1,D,0,NaN,NaN,

In [154]:
# Verificando linhas e colunas
df.shape

(46522, 203)

In [94]:
# Resumo Estatístico dos Dados
df.describe()

,index,alvo,DIAS_ATRASO,col_4,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,col_50,col_51,col_52,col_53,col_54,col_55,col_56,col_57,col_58,col_59,col_60,col_61,col_62,col_63,col_64,col_65,col_66,col_67,col_68,col_69,col_70,col_71,col_72,col_73,col_74,col_75,col_76,col_77,col_78,col_87,col_88,col_89,col_90,col_91,col_92,col_93,col_94,col_95,col_96,col_97,col_98,col_99,col_100,col_101,col_102,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,col_112,col_113,col_114,col_115,col_116,col_123,col_124,col_125,col_126,col_127,col_128,col_129,col_130,col_131,col_132,col_133,col_134,col_135,col_136,col_137,col_138,col_139,col_140,col_141,col_142,col_150,col_151,col_156,col_160,col_164,col_165,col_166,col_167,col_169,col_170,col_173,col_174,col_175,col_176,col_177,col_179,col_182,col_183,col_187,col_188,col_189,col_190,col_191,col_192,col_193,col_197,col_198,col_199
count,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,41769.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,42989.00,42989.00,41712.00,41712.00,14311.00,14311.00,37003.00,11520.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,20881.00,22587.00,18812.00,34945.00,10828.00,30530.00,11692.00,19311.00,3302.00,46522.00,46522.00,46522.00,45979.00,40402.00,46522.00,46522.00,45979.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,46522.00,42393.00,42374.00,43038.00
mean,23260.50,0.13,45.23,5705.32,0.27,0.00,0.00,0.00,0.02,0.14,0.00,0.00,0.17,0.60,0.01,0.00,0.00,0.03,0.27,0.00,0.00,0.30,1.10,0.01,0.00,0.00,0.06,0.53,0.00,0.00,0.51,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.60,1.20,2.21,0.87,2.01,4.35,7.69,0.29,0.63,1.31,2.59,0.18,0.51,1.26,2.16,0.40,0.87,1.77,2.94,0.23,0.47,1.02,2.35,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.27,0.52,0.74,0.00,0.19,0.41,0.01,0.34,0.02,0.00,0.01,0.02,0.73,0.00,0.27,0.01,0.00,0.00,0.01,0.17,0.72,4.43,46.18,8.67,43.58,10.25,36.27,14.15,7.28,5.28,2.64,0.68,0.01,0.03,5.04,3.09,0.81,0.01,0.03,5.64,3.80,1.07,0.01,0.04,6.65,0.20,0.59,0.62,0.49,0.64,10.13,14.87,3.57,8.01,0.32,0.00,0.04,0.07,0.26,0.17,0.22,0.75,0.12,0.00,0.54,0.07,0.00,0.02,0.41,0.09,0.04,0.19,0.00,0.00,2.43,2.94,-0.80,0.69
std,13429.89,0.33,8.36,14505.09,0.62,0.06,0.00,0.01,0.13,0.60,0.00,0.03,0.51,0.97,0.09,0.00,0.01,0.18,0.82,0.00,0.04,0.69,1.31,0.11,0.01,0.05,0.23,1.23,0.01,0.05,0.91,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.14,1.67,2.35,2.99,4.97,8.62,13.94,1.52,2.14,3.47,6.58,0.66,1.28,2.39,3.33,2.29,3.80,6.47,10.08,1.05,1.65,2.79,5.46,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.07,0.44,0.50,0.44,0.00,0.39,0.49,0.11,0.47,0.14,0.00,0.11,0.14,0.45,0.06,0.44,0.08,0.00,0.00,0.08,0.38,0.45,2.75,24.28,11.69,24.82,13.24,20.63,15.26,9.55,6.62,2.15,1.24,0.08,0.17,3.36,2.60,1.49,0.08,0.18,3.90,3.17,1.95,0.09,0.20,4.66,0.34,0.43,0.39,0.38,0.37,1

In [95]:
# Verificar o tipo de dados de cada variável
df.dtypes

index            int64
alvo             int64
REF_DATE        object
PRODUTO         object
DIAS_ATRASO      int64
col_4          float64
col_5           object
col_6            int64
col_7            int64
col_8            int64
col_9            int64
col_10           int64
col_11           int64
col_12           int64
col_13           int64
col_14           int64
col_15           int64
col_16           int64
col_17           int64
col_18           int64
col_19           int64
col_20           int64
col_21           int64
col_22           int64
col_23           int64
col_24           int64
col_25           int64
col_26           int64
col_27           int64
col_28           int64
col_29           int64
col_30           int64
col_31           int64
col_32           int64
col_33           int64
col_34           int64
col_35           int64
col_36           int64
col_37           int64
col_38           int64
col_39           int64
col_40           int64
col_41           int64
col_42     

In [96]:
# Verificando informações do arquivo
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46522 entries, 0 to 46521
Columns: 203 entries, index to REGIAO
dtypes: float64(52), int64(103), object(48)
memory usage: 72.1+ MB


### Tratando dados vazios

In [97]:
df.isnull().sum()

index              0
alvo               0
REF_DATE           0
PRODUTO            0
DIAS_ATRASO        0
col_4              0
col_5              0
col_6              0
col_7              0
col_8              0
col_9              0
col_10             0
col_11             0
col_12             0
col_13             0
col_14             0
col_15             0
col_16             0
col_17             0
col_18             0
col_19             0
col_20             0
col_21             0
col_22             0
col_23             0
col_24             0
col_25             0
col_26             0
col_27             0
col_28             0
col_29             0
col_30             0
col_31             0
col_32             0
col_33             0
col_34             0
col_35             0
col_36             0
col_37             0
col_38             0
col_39             0
col_40             0
col_41             0
col_42             0
col_43             0
col_44             0
col_45             0
col_46       

In [157]:
# Remoção de dados que tem mais de 27 mil de vazios
df = df.drop(columns=['col_113', 'col_114', 'col_116', 'col_120', 'col_121', 'col_122', 'col_142', 'col_151', 'col_160', 'col_180', 'col_181', 'col_186'])

KeyError: "['col_113', 'col_114', 'col_116', 'col_120', 'col_121', 'col_122', 'col_142', 'col_151', 'col_160', 'col_180', 'col_181', 'col_186'] not found in axis"

In [158]:
# df = df.rename(columns={'col_181': 'EMPREGO', 'col_184':'FAIXA_IDADE', 'col_5':'FAIXA_TEMPORAL'})
# df['EMPREGO'] = df['EMPREGO'].fillna('DESCONHECIDO')
df.isnull().sum()

index              0
alvo               0
REF_DATE           0
PRODUTO            0
DIAS_ATRASO        0
col_4              0
col_5              0
col_6              0
col_7              0
col_8              0
col_9              0
col_10             0
col_11             0
col_12             0
col_13             0
col_14             0
col_15             0
col_16             0
col_17             0
col_18             0
col_19             0
col_20             0
col_21             0
col_22             0
col_23             0
col_24             0
col_25             0
col_26             0
col_27             0
col_28             0
col_29             0
col_30             0
col_31             0
col_32             0
col_33             0
col_34             0
col_35             0
col_36             0
col_37             0
col_38             0
col_39             0
col_40             0
col_41             0
col_42             0
col_43             0
col_44             0
col_45             0
col_46       

In [159]:
df.head()

,index,alvo,REF_DATE,PRODUTO,DIAS_ATRASO,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,col_50,col_51,col_52,col_53,col_54,col_55,col_56,col_57,col_58,col_59,col_60,col_61,col_62,col_63,col_64,col_65,col_66,col_67,col_68,col_69,col_70,col_71,col_72,col_73,col_74,col_75,col_76,col_77,col_78,col_79,col_80,col_81,col_82,col_83,col_84,col_85,col_86,col_87,col_88,col_89,col_90,col_91,col_92,col_93,col_94,col_95,col_96,col_97,col_98,col_99,col_100,col_101,col_102,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,col_112,col_115,col_117,col_118,col_119,col_123,col_124,col_125,col_126,col_127,col_128,col_129,col_130,col_131,col_132,col_133,col_134,col_135,col_136,col_137,col_138,col_139,col_140,col_141,col_143,col_144,col_145,col_146,col_147,col_148,col_149,col_150,col_152,col_153,col_154,col_155,col_156,col_157,col_158,col_159,col_161,col_162,col_163,col_164,col_165,col_166,col_167,col_168,col_169,col_170,col_171,col_172,col_173,col_174,col_175,col_176,col_177,col_178,col_179,col_182,col_183,col_184,col_185,col_187,col_188,col_189,col_190,col_191,col_192,col_193,col_194,col_195,col_196,col_197,col_198,col_199,col_200,REGIAO
0,0,0,2023-01-31,CREDITO PESSOAL,52,2461.11,30 a 60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.00,0.00,2.00,2.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,A,A,C,C,A,A,A,A,0,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,7,44.65,7.65,44.65,7.65,18.50,INDEFINIDO,INDEFINIDO,INDEFINIDO,4,0,0,0,7,4,0,0,0,7,6,0,0,0,9,0.00,NaN,0.00,0.00,SEM APROVACAO,SEM PASSAGEM,SEM APROVACAO,SEM APROVACAO,SEM PASSAGEM,S,N,44.65,N,N,VAREJO,S,35.70,N,VAREJO-BANCOS,N,N,APROV:S - NOVASOLIC:S - SOLICMESMOLOCAL:N,APROV:N - NOVASOLIC:N - SOLICMESMOLOCAL:N,0,0,0,0.00,SEM INFORMACAO,0.00,0,FUNDAMENTAL COMPLETO,BASICO,0,1.00,0,0,1,E,0,0,0,DE 25 A 39 ANOS,Y,0,0,0,0,0,0,2,NaN,MEDIO,NaN,3.00,-1.00,1.00,URBANO,NORTE
1,1,0,2022-12-29,CREDITO PESSOAL,37,4674.50,30 a 60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,A,A,A,C,A,A,A,C,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,7,41.61,15.47,41.61,15.47,6.50,INDEFINIDO,INDEFINIDO,INDEFINIDO,3,2,0,0,7,3,2,0,0,7,3,2,0,0,7,NaN,NaN,0.67,0.67,SEM PASSAGEM,SEM PASSAGEM,SEM PASSAGEM,SEM PASSAGEM,SEM PASSAGEM,S,S,15.47,N,N,VAREJO,N,NaN,N,NaN,N,N,APROV:S - NOVASOLIC:N - SOLICMESMOLOCAL:N,APROV:S - NOVASOLIC:N - SOLICMESMOLOCAL:N,0,0,0,0.00,2 A 4 SM,1.00,1,SUPERIOR INCOMPLETO,MEDIO,0,1.00,0,0,1,D,0,0,0,DE 25 A 39 ANOS,Y,0,1,0,0,0,0,4,D,MEDIO,D,2.00,1.00,1.00,URBANO,NORDESTE
2,2,0,2023-09-29,CREDITO PESSOAL,36,43649.17,30 a 60,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,B,B,B,B,A,A,A,A,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,5,69.55,0.82,69.55,0.82,4.00,INDEFINIDO,INDEFINIDO,INDEFINIDO,2,2,0,0,6,2,2,0,0,6,2,2,0,0,6,1.00,NaN,NaN,1.00,ALTA,SEM PASSAGEM,SEM PASSAGEM,ALTA,SEM PASSAGEM,S,S,0.82,S,N,BANCOS,N,NaN,N,NaN,N,N,APROV:S - NOVASOLIC:N - SOLICMESMOLOCAL:N,APROV:S - NOVASOLIC:N - SOLICMESMOLOCAL:N,1,0,0,0.00,4 A 10 SM,1.00,1,SUPERIOR COMPLETO,ALTO,1,4.00,1,0,1,D,0,0,0,DE 40 A 59 ANOS,X,0,0,1,1,0,0,8,B,MUITO ALTO,B+,4.00,3.00,1.00,URBANO,SUDESTE
3,3,1,2023-02-28,CREDITO PESSOAL,48,1090.58,30 a 60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.

In [160]:
unique_values = []
for i in df.columns.tolist():
    print(i, ':', len(df[i].astype(str).value_counts()))
    unique_values.append(len(df[i].astype(str).value_counts()))

index : 46522
alvo : 2
REF_DATE : 13
PRODUTO : 1
DIAS_ATRASO : 30
col_4 : 44434
col_5 : 1
col_6 : 9
col_7 : 3
col_8 : 1
col_9 : 2
col_10 : 2
col_11 : 6
col_12 : 1
col_13 : 2
col_14 : 10
col_15 : 12
col_16 : 3
col_17 : 1
col_18 : 2
col_19 : 3
col_20 : 6
col_21 : 1
col_22 : 2
col_23 : 12
col_24 : 14
col_25 : 3
col_26 : 2
col_27 : 2
col_28 : 3
col_29 : 10
col_30 : 2
col_31 : 2
col_32 : 13
col_33 : 1
col_34 : 1
col_35 : 1
col_36 : 1
col_37 : 1
col_38 : 1
col_39 : 1
col_40 : 1
col_41 : 1
col_42 : 1
col_43 : 1
col_44 : 1
col_45 : 1
col_46 : 1
col_47 : 1
col_48 : 17
col_49 : 20
col_50 : 22
col_51 : 75
col_52 : 95
col_53 : 133
col_54 : 182
col_55 : 41
col_56 : 50
col_57 : 74
col_58 : 110
col_59 : 15
col_60 : 22
col_61 : 36
col_62 : 45
col_63 : 61
col_64 : 85
col_65 : 115
col_66 : 156
col_67 : 25
col_68 : 32
col_69 : 45
col_70 : 74
col_71 : 2
col_72 : 2
col_73 : 2
col_74 : 2
col_75 : 2
col_76 : 2
col_77 : 4
col_78 : 7
col_79 : 12
col_80 : 12
col_81 : 12
col_82 : 12
col_83 : 12
col_84 : 12
col_8

In [167]:
columns = 
columns

Index(['REF_DATE', 'PRODUTO', 'col_5', 'col_79', 'col_80', 'col_81', 'col_82',
       'col_83', 'col_84', 'col_85', 'col_86', 'col_117', 'col_118', 'col_119',
       'col_143', 'col_144', 'col_145', 'col_146', 'col_147', 'col_148',
       'col_149', 'col_152', 'col_153', 'col_154', 'col_155', 'col_157',
       'col_158', 'col_159', 'col_161', 'col_162', 'col_163', 'col_168',
       'col_171', 'col_172', 'col_178', 'col_184', 'col_185', 'col_194',
       'col_195', 'col_196', 'col_200', 'REGIAO'],
      dtype='object')

In [165]:
feature_names = ohe.get_feature_names_out(columns)
feature_names

array(['REF_DATE_2022-11-30', 'REF_DATE_2022-12-29',
       'REF_DATE_2023-01-31', 'REF_DATE_2023-02-28',
       'REF_DATE_2023-03-31', 'REF_DATE_2023-04-28',
       'REF_DATE_2023-05-31', 'REF_DATE_2023-06-30',
       'REF_DATE_2023-07-31', 'REF_DATE_2023-08-31',
       'REF_DATE_2023-09-29', 'REF_DATE_2023-10-31',
       'REF_DATE_2023-11-30', 'PRODUTO_CREDITO PESSOAL', 'col_5_30 a 60',
       'col_79_A', 'col_79_B', 'col_79_C', 'col_79_D', 'col_79_E',
       'col_79_F', 'col_79_G', 'col_79_H', 'col_79_I', 'col_79_J',
       'col_79_K', 'col_79_nan', 'col_80_A', 'col_80_B', 'col_80_C',
       'col_80_D', 'col_80_E', 'col_80_F', 'col_80_G', 'col_80_H',
       'col_80_I', 'col_80_J', 'col_80_K', 'col_80_nan', 'col_81_A',
       'col_81_B', 'col_81_C', 'col_81_D', 'col_81_E', 'col_81_F',
       'col_81_G', 'col_81_H', 'col_81_I', 'col_81_J', 'col_81_K',
       'col_81_nan', 'col_82_A', 'col_82_B', 'col_82_C', 'col_82_D',
       'col_82_E', 'col_82_F', 'col_82_G', 'col_82_H', 'col_82_I',

In [162]:
ohe = OneHotEncoder()

In [169]:
encoded_df = ohe.fit_transform(df[df.select_dtypes(include=['object']).columns])

df_encoded = pd.DataFrame(encoded_df, columns=feature_names)

ValueError: Shape of passed values is (46522, 1), indices imply (46522, 272)

In [171]:
encoded_df.numpy

<46522x272 sparse matrix of type '<class 'numpy.float64'>'
	with 1953924 stored elements in Compressed Sparse Row format>

# Analise Exploratoria

# Criando Modelo